In [24]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt
import cv2

In [25]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [27]:
# Load the metadata
metadata_path = r"metadata.csv"

metadata = pd.read_csv(metadata_path)
metadata.isna().sum()

image_id            0
split               0
sat_image_path      0
mask_path         343
dtype: int64

In [28]:
metadata.head()

,image_id,split,sat_image_path,mask_path
0,100694,train,train/100694_sat.jpg,train/100694_mask.png
1,102122,train,train/102122_sat.jpg,train/102122_mask.png
2,10233,train,train/10233_sat.jpg,train/10233_mask.png
3,103665,train,train/103665_sat.jpg,train/103665_mask.png
4,103730,train,train/103730_sat.jpg,train/103730_mask.png


In [29]:
metadata.split.unique()

array(['train', 'valid', 'test'], dtype=object)

In [31]:
class_dict_path = r"class_dict.csv"
class_dict = pd.read_csv(class_dict_path)
class_dict

,name,r,g,b
0,urban_land,0,255,255
1,agriculture_land,255,255,0
2,rangeland,255,0,255
3,forest_land,0,255,0
4,water,0,0,255
5,barren_land,255,255,255
6,unknown,0,0,0


In [32]:
class_dict.dtypes

name    object
r        int64
g        int64
b        int64
dtype: object

In [33]:
# Create a mapping from (r, g, b) to class index
class_map = {(row["r"], row["g"], row["b"]): idx for idx, row in class_dict.iterrows()}
class_map

{(0, 255, 255): 0,
 (255, 255, 0): 1,
 (255, 0, 255): 2,
 (0, 255, 0): 3,
 (0, 0, 255): 4,
 (255, 255, 255): 5,
 (0, 0, 0): 6}

In [34]:
metadata_train = metadata[metadata["split"] == "train"]
metadata_train

,image_id,split,sat_image_path,mask_path
0,100694,train,train/100694_sat.jpg,train/100694_mask.png
1,102122,train,train/102122_sat.jpg,train/102122_mask.png
2,10233,train,train/10233_sat.jpg,train/10233_mask.png
3,103665,train,train/103665_sat.jpg,train/103665_mask.png
4,103730,train,train/103730_sat.jpg,train/103730_mask.png
...,...,...,...,...
798,992507,train,train/992507_sat.jpg,train/992507_mask.png
799,994520,train,train/994520_sat.jpg,train/994520_mask.png
800,995492,train,train/995492_sat.jpg,train/995492_mask.png
801,997521,train,train/997521_sat.jpg,train/997521_mask.png


In [35]:
# Initialize lists to hold images and masks
images = []
masks = []

# Set image size
img_size = (128, 128)


for idx, row in metadata_train.iterrows():
    img_path = row["sat_image_path"]
    mask_path = row["mask_path"]

    img = Image.open(img_path).resize(img_size)
    images.append(np.array(img))

    mask = Image.open(mask_path).resize(img_size, Image.NEAREST)
    mask_array = np.array(mask)

    class_indices = np.zeros((mask_array.shape[0], mask_array.shape[1]), dtype=np.uint8)
    for color, idx in class_map.items():
        color = np.array(color)  # Convert to numpy array
        match = np.all(mask_array == color, axis=-1)
        class_indices[match] = idx

    masks.append(class_indices)

In [36]:
# Normalize pixels to 0 to 1 range
# Convert images and masks to numpy arrays
images = np.array(images) / 255.0
masks = np.array(masks)

# Add channel dimension to make it equal to image dimensions or it won't work for combined processing inside model. i think.
masks = np.expand_dims(masks, axis=-1)  

In [37]:
# Split data into training and validation sets
x_train, x_test, y_train, y_test = train_test_split(images, masks, test_size=0.1, random_state=42)

In [38]:
num_classes = 7

In [39]:
# # U-Net Model with VGG16 Encoder
# base_model = VGG16(include_top=False, weights="imagenet", input_shape=(128, 128, 3))

# # Encoder
# x1 = base_model.get_layer("block1_conv2").output
# x2 = base_model.get_layer("block2_conv2").output
# x3 = base_model.get_layer("block3_conv3").output
# x4 = base_model.get_layer("block4_conv3").output

# # Bottleneck
# b = Conv2D(512, (3, 3), activation="relu", padding="same")(x4)
# b = Conv2D(512, (3, 3), activation="relu", padding="same")(b)

# # Decoder
# u1 = UpSampling2D((2, 2))(b)
# u1 = concatenate([u1, x3])
# u1 = Conv2D(256, (3, 3), activation="relu", padding="same")(u1)
# u1 = Conv2D(256, (3, 3), activation="relu", padding="same")(u1)

# u2 = UpSampling2D((2, 2))(u1)
# u2 = concatenate([u2, x2])
# u2 = Conv2D(128, (3, 3), activation="relu", padding="same")(u2)
# u2 = Conv2D(128, (3, 3), activation="relu", padding="same")(u2)

# u3 = UpSampling2D((2, 2))(u2)
# u3 = concatenate([u3, x1])
# u3 = Conv2D(64, (3, 3), activation="relu", padding="same")(u3)
# u3 = Conv2D(64, (3, 3), activation="relu", padding="same")(u3)

# outputs = Conv2D(num_classes, (1, 1), activation="softmax")(u3)

# model = Model(inputs=base_model.input, outputs=outputs)

# # Compile model
# model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=["accuracy"])

# # Define callbacks
# callbacks = [
#     ModelCheckpoint("unet_model1.keras", save_best_only=True, monitor="val_loss"),
#     EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
# ]

In [40]:
# # Train the model
# history = model.fit(
#     x_train,
#     y_train,
#     epochs=10,
#     batch_size=8,
#     validation_split=0.1,
#     callbacks=callbacks,
# )

In [41]:
# Load the saved model
model = tf.keras.models.load_model("green_model2.h5")

In [42]:
# Predict on validation set
predictions = model.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step


In [43]:
# Select the fifth image's predictions
single_image_prediction = predictions[5]

# Define class indices for green space
forest_class_index = 3
agriculture_class_index = 1

# Calculate green space for the selected image
green_pixels = np.sum(
    (np.argmax(single_image_prediction, axis=-1) == forest_class_index)
    | (np.argmax(single_image_prediction, axis=-1) == agriculture_class_index)
)

# Calculate the total number of pixels for the selected image
total_pixels = single_image_prediction.shape[0] * single_image_prediction.shape[1]

# Calculate green coverage percentage for the selected image
green_coverage_percent = (green_pixels / total_pixels) * 100

print(f"Green Coverage for the selected image: {green_coverage_percent}%")

Green Coverage for the selected image: 27.044677734375%


In [44]:
# Check unique class predictions
unique_classes = np.unique(np.argmax(single_image_prediction, axis=-1))
print("Unique predicted classes in the image:", unique_classes)

Unique predicted classes in the image: [0 1 2 3]
